# Data Preparation

## Packages

In [2]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def chunk_ticks_rolling(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    
    temp_df['tick_avg'] = df['tick'].rolling(window=number_of_ticks).mean()
    temp_df['spread_avg'] = df['spread'].rolling(window=number_of_ticks).mean()
    temp_df['tick_sd'] = df['spread'].rolling(window=number_of_ticks).mean()
    
    temp_df = temp_df.dropna()
    
    return(temp_df)

## Parameters

In [5]:
year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')


number_of_ticks = 20
pip_diff = 0.00001
rsi_window = 14
rs_max = 1e6
window_s_ma = 20
window_l_ma = 50

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [6]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 713 ms


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


## Data manipulation

In [7]:
%%time
df = chunk_ticks(df, number_of_ticks)
#df = chunk_ticks_rolling(df, number_of_ticks)

df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')
df = pd.read_csv(chunk_file_path)
df.head()

100%|███████████████████████████████████████| 50000/50000 [00:11<00:00, 4452.39it/s]


Records : 50000
Wall time: 11.6 s


,tick_avg,spread_avg,tick_sd
0,1.200982,0.000382,0.000245
1,1.200657,0.000704,0.000092
2,1.200720,0.000756,0.000032
3,1.200684,0.000365,0.000109
4,1.200634,0.000248,0.000075


In [8]:
%%time
# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Moving Averages ------------------
df['sema'] = df['tick_avg'].ewm(span=window_s_ma).mean()
df['sema_diff'] = df['sema'].diff()
df['lema'] = df['tick_avg'].ewm(span=window_l_ma).mean()
df['lema_diff'] = df['lema'].diff()
df['ema_diff'] = df['sema'] - df['lema']

df['ssma'] = df['tick_avg'].rolling(window=window_s_ma).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=window_l_ma).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=window_s_ma).max()
df['min_tick'] = df['tick_avg'].rolling(window=window_s_ma).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

# Slopes -----------------------------
#df['small_sema_slope'] = df['sema'].rolling(window=window_s_ma).progress_apply(get_slope_s)
#df['long_sema_slope'] = df['sema'].rolling(window=window_l_ma).progress_apply(get_slope_l)

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > pip_diff] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -pip_diff] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

Total records : 49950
Wall time: 80.2 ms


## Write data to csv

In [9]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 1.59 s


## Print Report

df = df[['sema_diff',
'min_gap',
'max_gap',
'lema_diff',
'ssma_diff',
'rsi',
'rs',
'avg_loss',
'avg_gain',
'tick_sd',
'diff',
'direction']]

In [10]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       23557   47.161161
increase   13360   26.746747
decrease   13033   26.092092


,tick_avg,spread_avg,tick_sd,diff,avg_gain,avg_loss,rs,rsi,sema,sema_diff,...,ssma,ssma_diff,lsma,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,direction
0,1.201482,0.000177,0.000002,0.000006,0.000024,0.000012,1.974589,66.381910,1.201353,0.000014,...,1.201405,3.457500e-05,1.201034,0.000010,0.000371,1.201534,1.200890,0.000052,-0.000592,increase
1,1.201494,0.000165,0.000010,0.000012,0.000024,0.000012,2.014948,66.831929,1.201367,0.000014,...,1.201435,3.021250e-05,1.201051,0.000017,0.000384,1.201534,1.201273,0.000040,-0.000221,increase
2,1.201499,0.000148,0.000020,0.000005,0.000011,0.000012,0.886398,46.988906,1.201379,0.000013,...,1.201447,1.128750e-05,1.201067,0.000016,0.000380,1.201534,1.201319,0.000035,-0.000180,same
3,1.201461,0.000141,0.000026,-0.000038,0.000010,0.000015,0.653706,39.529758,1.201387,0.000008,...,1.201452,5.875000e-06,1.201082,0.000016,0.000370,1.201534,1.201319,0.000074,-0.000142,same
4,1.201332,0.000210,0.000022,-0.000128,0.000010,0.000022,0.445364,30.813288,1.201382,-0.000005,...,1.201452,-7.500000e-08,1.201096,0.000014,0.000356,1.201534,1.201319,0.000202,-0.000014,same


In [11]:
np.round((df['rsi']),0).value_counts()

49.0    925
44.0    920
56.0    902
50.0    889
48.0    885
       ... 
1.0      54
96.0     53
98.0     52
3.0      44
99.0     42
Name: rsi, Length: 101, dtype: int64

In [12]:
list(df.columns)

['tick_avg',
 'spread_avg',
 'tick_sd',
 'diff',
 'avg_gain',
 'avg_loss',
 'rs',
 'rsi',
 'sema',
 'sema_diff',
 'lema',
 'lema_diff',
 'ema_diff',
 'ssma',
 'ssma_diff',
 'lsma',
 'lsma_diff',
 'sma_diff',
 'max_tick',
 'min_tick',
 'max_gap',
 'min_gap',
 'direction']